# Primer on Stable Diffusion V2

Let's explore the stages involved in the Stable Diffusion V2 pre-training pipeline.

<div class="alert alert-block alert-info">
<b> Here is the roadmap for this notebook:</b>
<ul>
    <li><b>Part 1:</b> Pre-training of a Stable Diffusion Model</li>
    <li><b>Part 2:</b> Data pre-processing in more detail</li>
    <li><b>Part 3:</b> Compute requirements for pre-processing and training</li>
</ul>
</div>

Note you don't need to understand the architecture specifics of each component to build the pre-training pipeline. 

## 1. Pre-training of a Stable Diffusion V2 Model

Below is a diagram of the data flow in the pre-training of the Stable Diffusion V2 model. 

<figure>
  <img src="https://images.ctfassets.net/xjan103pcp94/4tVqmJbARiNxB0ym3vkuQU/0af145dbf213c85671ad1e5f761a1c15/image2.png" alt="Pre-training data flow of Stable Diffusion Model" width="800"/>
  <figcaption>Image taken from <a href="https://www.anyscale.com/blog/scalable-and-cost-efficient-stable-diffusion-pre-training-with-ray#stable-diffusion-primer">Reducing the Cost of Pre-training Stable Diffusion by 3.7x with Anyscale
</a></figcaption>
</figure>

As shown in the diagram, the pre-training of the Stable Diffusion V2 model consists of the following steps:

1. A pre-trained VAE and a text encoder(OpenCLIP-ViT/H) encodes the input images and text prompts. 
2. A trainable U-Net model learns the diffusion process with the image latents and text embeddings. 
3. The loss is calculated based on the input noise and the noise predicted by the U-Net.

Here's a visual representation of the full forward diffusion process as presented in the paper titled Denoising Diffusion Probabilistic Models.

<img src="https://images.ctfassets.net/xjan103pcp94/7wz6hsqYK9T4TBVdVNITvO/1d7e6c39c985ed998213c446233e5f60/image7.png" width="600"/>

The U-Net model improves at predicting and removing noise from images using text descriptions. This iterative process, involving noise prediction and subtraction, ultimately yields an image matching the text input.

Below is the reverse diffusion process visualized, which generates the final image:

<img src="https://images.ctfassets.net/xjan103pcp94/6nNd9J8cJu9gvBhRJ1mHkf/6e46032a5ea0167d577487cb7d19017a/image6.png" width="600"/>



## 2. Data pre-processing in more detail

### Encoding the input images and text prompts

Below is a diagram showing the how the VAE encodes/decodes the input images.

<figure>
  <img src="https://images.ctfassets.net/xjan103pcp94/6Pggjim01hWDxBxlV0kcmR/aa6ef7f6f859cfb0d864f48021cd9ef9/image12.png" width="600"/>
  <figcaption>Image taken from <a href="https://www.anyscale.com/blog/processing-2-billion-images-for-stable-diffusion-model-training-definitive-guides-with-ray-series">Processing 2 Billion Images for Stable Diffusion Model Training
</a></figcaption>

Basically a VAE model will:
- Encode the input image into a latent space think of it as a compressed representation of the input image.
- Decode the latent space back to the original image.


Below is a diagram showing the how the text encoder encodes the input text prompts.

<figure>
  <img src="https://images.ctfassets.net/xjan103pcp94/6esOzFJAsnffQDV7lYMxMG/cebf0af6d282e988149f2af4cba72479/image8.png" width="600"/>
  <figcaption>Image taken from <a href="https://www.anyscale.com/blog/processing-2-billion-images-for-stable-diffusion-model-training-definitive-guides-with-ray-series">Processing 2 Billion Images for Stable Diffusion Model Training
</a></figcaption>
</figure>

Basically a text encoder model will:
- Tokenize the input text prompt given a particular vocabulary and convert it to a sequence of tokens.
- Encode the sequence of tokens into a latent space think of it as a compressed vector representation of the input text prompt.

## 3. Compute requirements for pre-processing and training

Running the encoders on the same GPU as the U-Net model is not efficient. 

The encoders are smaller models and won't be able to use the large VRAM on the A100 GPUs if the same batch size needs to be used across all the models.

Instead, we would like a heterogeneous cluster of machines where we use A10G GPUs for the encoders and larger A100 GPUs for the U-Net model.

<img src="https://anyscale-public-materials.s3.us-west-2.amazonaws.com/ray-summit/stable-diffusion/diagrams/resource_annotated_training_v2.png" width="800"/>

